## 실시간 엔드포인트를 DJL Serving을 이용해 호스팅
이번 노트북에서는 [Deep Java Library](https://djl.ai/) (DJServing)가 지원하는 대형 모델 추론 컨테이너(LMI)를 모델 서빙 솔루션으로 사용합니다. 

DJL Serving은 별도의 사전 컴파일 없이도 HuggingFace SafeTensor 형식의 모델 웨이트를 자동으로 컴파일하고, 로드할 수 있도록 해 줍니다.

AWS Neuron SDK는 사용자가 Inferentia 장치의 강력한 처리 능력을 쉽게 활용할 수 있게 해주며, DJLServing은 Java 기반 환경에서 대규모 모델을 손쉽게 서빙할 수 있도록 지원합니다. 이 노트북은 Amazon Elastic Compute Cloud(Amazon EC2) inf2.xlarge 인스턴스에 Llama 3 모델을 배포합니다.

In [84]:
# 원하는 리전으로 변경합니다.
aws_region = "ap-northeast-1"

# 컨테이너 이미지 확인
image_uri = f"763104351884.dkr.ecr.{aws_region}.amazonaws.com/djl-inference:0.28.0-neuronx-sdk2.18.2"
image_uri

'763104351884.dkr.ecr.ap-northeast-1.amazonaws.com/djl-inference:0.28.0-neuronx-sdk2.18.2'

In [85]:
from huggingface_hub import login, snapshot_download

# Hugging Face 로그인
login()

In [87]:
# 모델 다운로드
model_id = "meta-llama/Meta-Llama-3-8B"
local_model_path = "./models/llama3"
snapshot_download(repo_id=model_id, local_dir=local_model_path)

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

'/home/ubuntu/inferentia2-llm/models/llama3'

In [166]:
# 모델 서버에 필요한 serving.properties 파일 생성 (로컬 경로 참조)

file_content = f"""engine=Python
option.entryPoint=djl_python.transformers_neuronx
option.model_id=/opt/ml/model/llama3
option.n_positions=8192
option.rolling_batch=vllm
option.max_rolling_batch_size=16
option.tensor_parallel_degree=8
option.enable_mixed_precision_accumulation=true
option.load_split_model=false
option.group_query_attention=replicated-heads"""

with open("serving.properties","w") as f:
    f.write(file_content)

In [162]:
# # 모델 서버에 필요한 serving.properties 파일 생성 (로컬 경로 참조)

# file_content = f"""engine=Python
# option.entryPoint=djl_python.transformers_neuronx
# option.model_id=/opt/ml/model/llama3
# option.batch_size=1
# option.neuron_optimize_level=2
# option.load_in_8bit=false
# option.n_positions=8192
# option.rolling_batch=auto
# option.tensor_parallel_degree=12
# option.dtype=bf16
# option.fuse_qkv=true
# option.attention_layout=BSH
# option.group_query_attention=replicated-heads
# option.load_split_model=false"""

# with open("serving.properties","w") as f:
#     f.write(file_content)

# scheduler: If you set engine to python and set rolling_batch to be auto or scheduler, we will use this. This is our default batch implementation focusing on HuggingFace Accelerate use cases.
# lmi-dist: If you set engine to MPI and set rolling_batch to be auto or lmi-dist, we will use this. This is our default batch implementation focusing on tensor parallel use cases. It has flashAttention and PagedAttention implmentation, inspired from Text-Generation-Inference design
# vllm: If you set engine to Python and set rolling_batch to be vllm, we will use this. This is our batch implementation to run vllm from its library, you also need to have a requirements.txt installing vllm==0.1.7 and pandas to use it.

In [168]:
%%sh

# `serving.properties`를 models/llama3 디렉토리로 이동
mkdir -p logs
mkdir -p neuron-cache
cp serving.properties models/llama3/

In [169]:
!aws ecr get-login-password --region ap-northeast-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.ap-northeast-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [ ]:
!docker run -it --rm --network=host \
  -v $(pwd)/models:/opt/ml/model/ \
  -v $(pwd)/logs:/opt/djl/logs \
  -v $(pwd)/neuron-cache:/var/tmp/neuron-compile-cache \
  -u djl \
  --device /dev/neuron0 \
  --device /dev/neuron1 \
  --device /dev/neuron2 \
  --device /dev/neuron3 \
  --device /dev/neuron4 \
  --device /dev/neuron5 \
  -e MODEL_LOADING_TIMEOUT=7200 \
  -e PREDICT_TIMEOUT=360 \
  {image_uri} serve

INFO  Ec2Utils DJL will collect telemetry to help us better understand our users? needs, diagnose issues, and deliver additional features. If you would like to learn more or opt-out please go to: https://docs.djl.ai/docs/telemetry.html for more information.
INFO  ModelServer Starting model server ...
INFO  ModelServer Starting djl-serving: 0.28.0 ...
INFO  ModelServer 
Model server home: /opt/djl
Current directory: /opt/djl
Temp directory: /tmp
Command line: -Dlog4j.configurationFile=/usr/local/djl-serving-0.28.0/conf/log4j2.xml -Xmx1g -Xms1g -Xss2m -XX:+ExitOnOutOfMemoryError
Number of CPUs: 96
Number of Neuron cores: 12
Max heap size: 1024
Config file: /opt/djl/conf/config.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Default job_queue_size: 1000
Default batch_size: 1
Default max_batch_delay: 100
Default max_idle_time: 60
Model Store: /opt/ml/model
Initial Models: ALL
Netty threads: 0
Maximum Request Size: 67108864
Environment variables:
  

## 추론 테스트
Docker 엔드포인트가 생성된 후, Predictor 객체를 사용하여 Docker 엔드포인트에 대해 실시간 예측을 수행할 수 있습니다.
- 추론 요청을 제출하고 응답을 받기 위해 `curl` 명령어를 사용합니다.
- 요청과 응답은 JSON 형식으로 이루어집니다.

아래 추론 코드를 별도의 터미널을 열고 실행합니다.

In [ ]:
curl -N -X POST "http://127.0.0.1:8080/predictions/llama3" \
     -H 'Content-Type: application/json' \
     -d '{
         "seq_length": 512,
         "inputs": "Welcome to Amazon Elastic Compute Cloud",
         "parameters": {
             "max_new_tokens": 32,
             "do_sample": "true"
         }
     }'